[Reference](https://towardsdev.com/fastapi-from-app-py-to-a-modular-architecture-54ca9e0044eb)

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

# Pydantic model for creating a new todo
class TodoCreate(BaseModel):
    title: str

# Pydantic model for a todo item, inherits from TodoCreate and adds id and completed fields
class Todo(TodoCreate):
    id: int
    completed: bool = False

# In-memory storage for todos (simulating a database)
todos = []

# Endpoint to create a new todo
@app.post("/todos", response_model=Todo)
def create_todo(todo: TodoCreate):
    # Create a new todo item with an incremented id
    new_todo = Todo(id=len(todos) + 1, **todo.model_dump())
    todos.append(new_todo)  # Add the new todo to the list
    return new_todo  # Return the created todo as response

# Endpoint to fetch all todos
@app.get("/todos", response_model=list[Todo])
def get_todos():
    return todos  # Return the list of todos as response

# Endpoint to fetch a specific todo by its id
@app.get("/todos/{todo_id}", response_model=Todo)
def get_todo(todo_id: int):
    for todo in todos:
        if todo.id == todo_id:
            return todo  # Return the todo if found
    # Raise HTTPException with 404 status code and message if todo is not found
    raise HTTPException(status_code=404, detail="Todo not found")

# Endpoint to update a todo by its id
@app.put("/todos/{todo_id}", response_model=Todo)
def update_todo(todo_id: int, updated_todo: TodoCreate):
    for todo in todos:
        if todo.id == todo_id:
            todo.title = updated_todo.title  # Update the title of the todo
            return todo  # Return the updated todo
    # Raise HTTPException with 404 status code and message if todo is not found
    raise HTTPException(status_code=404, detail="Todo not found")

# Endpoint to delete a todo by its id
@app.delete("/todos/{todo_id}")
def delete_todo(todo_id: int):
    for index, todo in enumerate(todos):
        if todo.id == todo_id:
            del todos[index]  # Delete the todo from the list
            return {"message": "Todo deleted successfully"}  # Return success message
    # Raise HTTPException with 404 status code and message if todo is not found
    raise HTTPException(status_code=404, detail="Todo not found")

# Main block to run the application using Uvicorn server
if __name__ == "__main__":
    import uvicorn
    uvicorn.run("app:app", port=3000, host="0.0.0.0", reload=True)

In [2]:
from fastapi import APIRouter

router = APIRouter()

@router.post("/todos")
def create_todo():
    pass

@router.get("/todos")
def get_todos():
    pass

@router.get("/todos/{todo_id}")
def get_todo(todo_id: int):
    pass

@router.put("/todos/{todo_id}")
def update_todo(todo_id: int):
    pass

@router.delete("/todos/{todo_id}")
def delete_todo(todo_id: int):
    pass

In [3]:
from fastapi import FastAPI
from routers import todo_router

app = FastAPI()

app.include_router(todo_router.router)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("app:app", port=3000, host="0.0.0.0", reload=True)

In [4]:
from fastapi import HTTPException
from pydantic import BaseModel

class TodoCreate(BaseModel):
    title: str

class Todo(TodoCreate):
    id: int
    completed: bool = False

class TodoController:
    def __init__(self):
        self.todos = []

    def create_todo(self, todo: TodoCreate):
        new_todo = Todo(id=len(self.todos) + 1, **todo.model_dump())
        self.todos.append(new_todo)
        return new_todo

    def get_todos(self):
        return self.todos

    def get_todo(self, todo_id: int):
        for todo in self.todos:
            if todo.id == todo_id:
                return todo
        raise HTTPException(status_code=404, detail="Todo not found")

    def update_todo(self, todo_id: int, updated_todo: TodoCreate):
        for todo in self.todos:
            if todo.id == todo_id:
                todo.title = updated_todo.title
                return todo
        raise HTTPException(status_code=404, detail="Todo not found")

    def delete_todo(self, todo_id: int):
        for index, todo in enumerate(self.todos):
            if todo.id == todo_id:
                del self.todos[index]
                return {"message": "Todo deleted successfully"}
        raise HTTPException(status_code=404, detail="Todo not found")

In [5]:
from fastapi import APIRouter
from controllers.todo_controller import TodoController, TodoCreate, Todo

router = APIRouter()
todo_controller = TodoController()

@router.post("/todos", response_model=Todo)
def create_todo(todo: TodoCreate):
    return todo_controller.create_todo(todo)

@router.get("/todos", response_model=list[Todo])
def get_todos():
    return todo_controller.get_todos()

@router.get("/todos/{todo_id}", response_model=Todo)
def get_todo(todo_id: int):
    return todo_controller.get_todo(todo_id)

@router.put("/todos/{todo_id}", response_model=Todo)
def update_todo(todo_id: int, updated_todo: TodoCreate):
    return todo_controller.update_todo(todo_id, updated_todo)

@router.delete("/todos/{todo_id}")
def delete_todo(todo_id: int):
    return todo_controller.delete_todo(todo_id)

In [6]:
from pydantic import BaseModel

class TodoCreate(BaseModel):
    title: str

class Todo(TodoCreate):
    id: int
    completed: bool = False

class TodoService:
    def __init__(self):
        self.todos = []

    def create_todo(self, todo: TodoCreate) -> Todo:
        new_todo = Todo(id=len(self.todos) + 1, **todo.model_dump())
        self.todos.append(new_todo)
        return new_todo

    def get_todos(self) -> list[Todo]:
        return self.todos

    def get_todo(self, todo_id: int) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                return todo
        return None

    def update_todo(self, todo_id: int, updated_todo: TodoCreate) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                todo.title = updated_todo.title
                return todo
        return None

    def delete_todo(self, todo_id: int) -> bool:
        for index, todo in enumerate(self.todos):
            if todo.id == todo_id:
                del self.todos[index]
                return True
        return False

In [7]:
from fastapi import HTTPException
from services.todo_service import TodoService, TodoCreate, Todo

class TodoController:
    def __init__(self):
        self.todo_service = TodoService()

    def create_todo(self, todo: TodoCreate):
        return self.todo_service.create_todo(todo)

    def get_todos(self):
        return self.todo_service.get_todos()

    def get_todo(self, todo_id: int):
        todo = self.todo_service.get_todo(todo_id)
        if todo is None:
            raise HTTPException(status_code=404, detail="Todo not found")
        return todo

    def update_todo(self, todo_id: int, updated_todo: TodoCreate):
        todo = self.todo_service.update_todo(todo_id, updated_todo)
        if todo is None:
            raise HTTPException(status_code=404, detail="Todo not found")
        return todo

    def delete_todo(self, todo_id: int):
        if self.todo_service.delete_todo(todo_id):
            return {"message": "Todo deleted successfully"}
        raise HTTPException(status_code=404, detail="Todo not found")

In [8]:
from pydantic import BaseModel

class TodoCreate(BaseModel):
    title: str

class Todo(TodoCreate):
    id: int
    completed: bool = False

class TodoRepository:
    def __init__(self):
        self.todos = []

    def create_todo(self, todo: TodoCreate) -> Todo:
        new_todo = Todo(id=len(self.todos) + 1, **todo.model_dump())
        self.todos.append(new_todo)
        return new_todo

    def get_todos(self) -> list[Todo]:
        return self.todos

    def get_todo(self, todo_id: int) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                return todo
        return None

    def update_todo(self, todo_id: int, updated_todo: TodoCreate) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                todo.title = updated_todo.title
                return todo
        return None

    def delete_todo(self, todo_id: int) -> bool:
        for index, todo in enumerate(self.todos):
            if todo.id == todo_id:
                del self.todos[index]
                return True
        return False

In [9]:
from pydantic import BaseModel

class TodoCreate(BaseModel):
    title: str

class Todo(TodoCreate):
    id: int
    completed: bool = False

class TodoRepository:
    def __init__(self):
        self.todos = []

    def create_todo(self, todo: TodoCreate) -> Todo:
        new_todo = Todo(id=len(self.todos) + 1, **todo.model_dump())
        self.todos.append(new_todo)
        return new_todo

    def get_todos(self) -> list[Todo]:
        return self.todos

    def get_todo(self, todo_id: int) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                return todo
        return None

    def update_todo(self, todo_id: int, updated_todo: TodoCreate) -> Todo | None:
        for todo in self.todos:
            if todo.id == todo_id:
                todo.title = updated_todo.title
                return todo
        return None

    def delete_todo(self, todo_id: int) -> bool:
        for index, todo in enumerate(self.todos):
            if todo.id == todo_id:
                del self.todos[index]
                return True
        return False